In [1]:
import pandas as pd
import os   
import numpy as np
import random


In [2]:
save_dir = 'data/ssusa'
os.makedirs(save_dir, exist_ok=True)

# Show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

In [3]:
# Define the directory path
data_dir = save_dir

# Define file paths
sequences_path = data_dir + r'\ssusa_finalsequences.csv'
deployments_path = data_dir + r'\ssusa_finaldeployments.csv'

# Load the CSV files
sequences_df = pd.read_csv(
    sequences_path,
    na_values=["NA", " "],
    dtype={
        "Year": "Int64",
        "Sequence_ID": "object",
        "Group_Size": "Int64",
    },
    keep_default_na=True,
    parse_dates=["Start_Time", "End_Time"]
)
print(f"Number of records in sequences_df: {len(sequences_df)}")

deployments_df = pd.read_csv(
    deployments_path,
    na_values=["NA", " "],
    dtype={
        "Year": "Int64",
        "Survey_Nights": "float64", 
        "Latitude": "float64",
        "Longitude": "float64",
    },
    keep_default_na=True,
    parse_dates=["Start_Date", "End_Date"]
)
print(f"Number of records in deployments_df: {len(deployments_df)}")

Number of records in sequences_df: 987979
Number of records in deployments_df: 9679


In [4]:
# List of required columns
required_cols = ['Class', 'Order', 'Family', 'Genus', 'Species', 'Common_Name']

# Replace single space strings with actual NaN
sequences_df[required_cols] = sequences_df[required_cols].replace(' ', pd.NA)

# Count before filtering
initial_count = len(sequences_df)

# Drop rows with any missing values in required columns
sequences_df = sequences_df.dropna(subset=required_cols)

# Count after filtering
final_count = len(sequences_df)
deleted_count = initial_count - final_count

# Print results
print(f"Number of records deleted: {deleted_count}")
print(f"Final number of records: {final_count}")

Number of records deleted: 96032
Final number of records: 891947


In [5]:
sequences_df.head(3)

,Year,Project,Camera_Trap_Array,Deployment_ID,Sequence_ID,Start_Time,End_Time,Class,Order,Family,Genus,Species,Common_Name,Age,Sex,Group_Size
0,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s1,2019-08-31 06:50:00,2019-08-31 06:50:00,Mammalia,Carnivora,Ursidae,Ursus,arctos,Brown Bear,Unknown,Unknown,1
1,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s2,2019-08-31 14:15:00,2019-08-31 14:17:00,Mammalia,Carnivora,Ursidae,Ursus,arctos,Brown Bear,Unknown,Unknown,1
2,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s3,2019-08-31 18:22:00,2019-08-31 18:22:00,Mammalia,Carnivora,Ursidae,Ursus,arctos,Brown Bear,Unknown,Unknown,1


In [6]:
# Add 'Sci_Name' column by combining 'Genus' and 'Species' (Genus capitalized, Species lowercase)
sequences_df['Sci_Name'] = sequences_df['Genus'].str.capitalize().str.strip() + \
                            ' ' + sequences_df['Species'].str.lower().str.strip()
sequences_df['Sci_Name'] = sequences_df['Sci_Name'].str.strip()

# move 'Sci_Name' column to be after Common_Name
sci_name_col = sequences_df.pop('Sci_Name')
common_name_index = sequences_df.columns.get_loc('Common_Name')
sequences_df.insert(common_name_index + 1, 'Sci_Name', sci_name_col)

In [7]:
# Get all common columns
common_cols = set(sequences_df.columns).intersection(deployments_df.columns)
print(common_cols)

# Merge on both 'Deployment_ID' and 'Year'
merged_df = pd.merge(
    sequences_df, 
    deployments_df, 
    on=list(common_cols), 
    how='inner'
)

merged_df = merged_df.drop_duplicates()

print(f"Number of records in merged_df: {len(merged_df)}")

{'Project', 'Year', 'Deployment_ID', 'Camera_Trap_Array'}
Number of records in merged_df: 885087


In [8]:
# Columns to convert to Proper Case
prop_case_cols = [
    'Class', 'Order', 'Family', 'Genus', 'Species', 'Habitat', 
    'Development_Level', 'Feature_Type', 'Common_Name', 'Age', 'Sex'
]

# First, replace blank or whitespace-only strings with NaN for Age, Sex, and Group_Size
merged_df[['Age', 'Sex', 'Group_Size']] = merged_df[['Age', 'Sex', 'Group_Size']].replace(r'^\s*$', pd.NA, regex=True)

# Group_Size: Replace missing with 0 and convert to numeric
merged_df['Group_Size'] = merged_df['Group_Size'].fillna(0)
merged_df['Group_Size'] = pd.to_numeric(merged_df['Group_Size'], errors='coerce').fillna(0).astype(int)

# Age and Sex: Replace missing with 'Unknown'
merged_df['Age'] = merged_df['Age'].fillna('Unknown')
merged_df['Sex'] = merged_df['Sex'].fillna('Unknown')

# Now apply proper case to selected columns
for col in prop_case_cols:
    merged_df[col] = merged_df[col].str.lower()

In [9]:
# Keep only rows where Class is 'Mammalia'
merged_df = merged_df[merged_df['Class'] == 'mammalia']
print(f"Number of records in merged_df after filtering for Mammalia: {len(merged_df)}")

Number of records in merged_df after filtering for Mammalia: 857003


In [10]:
# Drop all human data
merged_df = merged_df[
    (merged_df['Genus'].str.lower() != 'homo') &
    (merged_df['Species'].str.lower() != 'sapiens')
].reset_index(drop=True)
print(f"Number of records in merged_df after dropping human data: {len(merged_df)}")

Number of records in merged_df after dropping human data: 772478


In [11]:
# Find any possible domestic animals in merged_df by searching for keywords in 'Common_Name'
domestic_keywords = ['dog', 'cat', 'cattle', 'cow', 'horse', 'sheep', 'pig', 'goat', 'donkey', 'chicken', 'rooster', 'domestic', 'pet']

# Create a mask for rows where 'Common_Name' contains any domestic keyword
mask_domestic = merged_df['Common_Name'].str.contains('|'.join(domestic_keywords), case=False, na=False)

# Show all rows with possible domestic animals
domestic_animals_found = merged_df[mask_domestic].reset_index(drop=True)
print(f"Number of possible domestic animal records found: {len(domestic_animals_found)}")
domestic_animals_found[['Common_Name', 'Sci_Name', 'Genus', 'Species']].drop_duplicates().reset_index(drop=True)

Number of possible domestic animal records found: 74551


,Common_Name,Sci_Name,Genus,Species
0,domestic dog,Canis familiaris,canis,familiaris
1,domestic cat,Felis catus,felis,catus
2,bobcat,Lynx rufus,lynx,rufus
3,domestic cattle,Bos taurus,bos,taurus
4,wild pig,Sus scrofa,sus,scrofa
5,domestic horse,Equus caballus,equus,caballus
6,black-tailed prairie dog,Cynomys ludovicianus,cynomys,ludovicianus
7,domestic sheep,Ovis aries,ovis,aries
8,domestic donkey,Equus asinus,equus,asinus
9,bighorn sheep,Ovis canadensis,ovis,canadensis


In [12]:
# Drop all domestic species
domestic_common = [
    "domestic dog", "domestic cat", "domestic cattle", "domestic horse", 
    "domestic sheep", "domestic pig", "domestic goat", "domestic donkey"
]
merged_df = merged_df[~merged_df['Common_Name'].str.lower().isin(domestic_common)].reset_index(drop=True)
print(f"Number of records in merged_df after dropping domestic species: {len(merged_df)}")

Number of records in merged_df after dropping domestic species: 718137


In [13]:
# Show all rows with remaining domestic animals
mask_domestic = merged_df['Common_Name'].str.contains('|'.join(domestic_keywords), case=False, na=False)
domestic_animals_found = merged_df[mask_domestic].reset_index(drop=True)

print(f"Number of remaining possible domestic animal records found: {len(domestic_animals_found)}")
domestic_animals_found[['Common_Name', 'Sci_Name', 'Genus', 'Species']].drop_duplicates().reset_index(drop=True)

Number of remaining possible domestic animal records found: 20210


,Common_Name,Sci_Name,Genus,Species
0,bobcat,Lynx rufus,lynx,rufus
1,wild pig,Sus scrofa,sus,scrofa
2,black-tailed prairie dog,Cynomys ludovicianus,cynomys,ludovicianus
3,bighorn sheep,Ovis canadensis,ovis,canadensis
4,arizona black-tailed prairie dog,Cynomys ludovicianus,cynomys,ludovicianus
5,white-tailed prairie dog,Cynomys leucurus,cynomys,leucurus


In [14]:
# Check that each unique Common_Name in merged_df corresponds to only one unique Sci_Name
common_to_sciname_counts = merged_df.groupby('Common_Name')['Sci_Name'].nunique()
print(common_to_sciname_counts[common_to_sciname_counts > 1])
sciname_to_common_counts = merged_df.groupby('Sci_Name')['Common_Name'].nunique()
print(sciname_to_common_counts[sciname_to_common_counts > 1])

Series([], Name: Sci_Name, dtype: int64)
Sci_Name
Cynomys ludovicianus    2
Name: Common_Name, dtype: int64


In [15]:
# Find unique common name with sci name Cynomys ludovicianus
unique_common_names = merged_df[merged_df['Sci_Name'] == 'Cynomys ludovicianus']['Common_Name'].unique()
unique_common_names

array(['black-tailed prairie dog', 'arizona black-tailed prairie dog'],
      dtype=object)

In [16]:
# Combine all unique common names for Cynomys ludovicianus into a single standardized name
merged_df.loc[merged_df['Sci_Name'] == 'Cynomys ludovicianus', 'Common_Name'] = 'black-tailed prairie dog'

In [17]:
# Show updated information about the cleaned DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718137 entries, 0 to 718136
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Year               718137 non-null  Int64         
 1   Project            718137 non-null  object        
 2   Camera_Trap_Array  718137 non-null  object        
 3   Deployment_ID      718137 non-null  object        
 4   Sequence_ID        708175 non-null  object        
 5   Start_Time         718137 non-null  datetime64[ns]
 6   End_Time           718137 non-null  datetime64[ns]
 7   Class              718137 non-null  object        
 8   Order              718137 non-null  object        
 9   Family             718137 non-null  object        
 10  Genus              718137 non-null  object        
 11  Species            718137 non-null  object        
 12  Common_Name        718137 non-null  object        
 13  Sci_Name           718137 non-null  object  

In [18]:
# Optional: Save merged result to a new CSV
merged_df.to_csv(data_dir + r'\cleaned_snapshot_usa.csv', index=False)
print("Cleaned data saved to 'cleaned_snapshot_usa.csv'")

Cleaned data saved to 'cleaned_snapshot_usa.csv'
